In [ ]:
import uproot
import matplotlib.pyplot as plt
from scipy.interpolate import interp2d
import numpy as np
import healpy as hp
from tqdm import tqdm
import sys
import ROOT as rt
import root_numpy as rn

#root threeML astromodels numpy matplotlib healpy scipy astropy sympy uproot root_numpy 

In [ ]:
data_name = "~/home/data.root"
file = uproot.open(data_name)
histogram_on = file["all_sky_cube_on;1"]
histogram_off = file["all_sky_cube_bg;1"]
header = file["header"].title.split(",")
header_value=file["header"].values()
print(header,header_value)

In [ ]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt


# variogram_model是变差函数模型，pykrige提供 linear, power, gaussian, spherical, exponential, hole-effect几种variogram_model可供选择，默认的为linear模型。
# 使用不同的variogram_model，预测效果是不一样的，应该针对自己的任务选择合适的variogram_model。

k3d1, ss3d = ok3d.execute("grid", gridx, gridy) # k3d1是结果，给出了每个网格点处对应的值


print(np.round(k3d1,2))

# 绘图
fig, (ax1) = plt.subplots(1)
ax1.imshow(k3d1, origin="lower")
ax1.set_title("ordinary kriging")
plt.tight_layout()
plt.show()

In [ ]:
from pykrige.ok import OrdinaryKriging
skymap = histogram_on.values()[:,:,0].T
plt.imshow(skymap, origin="lower", extent=[0,360,-20,80], aspect=True)
skymapoff = histogram_off.values()[:,:,0].T
ra = histogram_on.axis("x").centers()
dec = histogram_on.axis("y").centers()
RA, DEC=np.meshgrid(ra,dec)
ok3d = OrdinaryKriging(RA[0:100,0:100].flatten(), DEC[0:100,0:100].flatten(), skymap[0:100,0:100].astype(np.int64).flatten(), variogram_model="linear")
k3d1, ss3d = ok3d.execute("grid", 80, 22) # 模型
# fon = interp2d(ra,dec,skymap,kind='linear')
# fbkg = interp2d(ra,dec,skymapoff,kind='linear')

In [ ]:
nside=1024
npix=hp.nside2npix(nside)
dtype1 = [('count', float)]
dtype2 = [('count', float)]
skymaponout=np.zeros(npix) #, dtype = dtype1
skymapoffout=np.zeros(npix) #, dtype = dtype2
pixid = np.arange(npix)
pixarea= 4*np.pi/npix
new_lats = 90-hp.pix2ang(nside, pixid)[0]*180/np.pi # thetas I need to populate with interpolated theta values
new_lons = hp.pix2ang(nside, pixid)[1]*180/np.pi # phis, same

ras = histogram_on.axis(0).centers()
decs = histogram_on.axis(1).centers()

for bin in range(14):
    print('nHit%02d'%int(bin))
    skymap = histogram_on.values()[:,:,bin].T
    skymapoff = histogram_off.values()[:,:,bin].T
    fbkg = interp2d(ras,decs,skymapoff,kind='linear')
    
    for j in range(len(decs)):
        dec=decs[j]
        if (dec>=-25 and dec<=85):
            with open("/data/home/cwy/Science/3MLWCDA/Standard/src/tools/LHAASO_skytrans/skytxt/sky_bin%i_on%i.txt"%(bin, j),"a+") as fs:
                for i in tqdm(range(len(ras))):
                    ra=ras[i]
                    pick = (new_lons>ra-0.05) & (new_lons<ra+0.05) & (new_lats>dec-0.05) & (new_lats<dec+0.05)
                    pixneed = pixid[pick]
                    lens = np.sum(pick)
                    if lens:
                        poissonrand = np.random.poisson(skymap[i][j]/lens, lens)
                        for k in range(lens):
                            ra_pix , dec_pix = hp.pix2ang(1024,pixneed[k],lonlat=True)
                            coff = fbkg(ra_pix,dec_pix)/(np.radians(0.1)*np.radians(0.1)*np.cos(np.radians(dec_pix)))*pixarea
                            fs.write(str(0)+" "+str(pixneed[k])+" "+str(poissonrand[k])+"\n")
                            fs.write(str(1)+" "+str(pixneed[k])+" "+str(coff)+"\n")

In [ ]:
# maptree2 = "../../data/KM2a_testnew.root"
# file2 = uproot.open(maptree2)
# file2.keys()
# for i in range(14):
#     data = file2[f"nHit{i:02d}/data;1"].arrays(library="np")
#     bkg = file2[f"nHit{i:02d}/bkg;1"].arrays(library="np")
#     data = hp.ma(data["count"])
#     bkg = hp.ma(bkg["count"])
#     # hp.mollview(data)
#     # hp.mollview(bkg)
#     # hp.mollview(data-bkg)
#     skymap = histogram_on.values()[:,:,i].T
#     skymapoff = histogram_off.values()[:,:,i].T
#     print(i, np.ma.sum(data)/np.sum(skymap), np.ma.sum(bkg)/np.sum(skymapoff))

In [ ]:
fout = rt.TFile.Open(f"./test.root", 'recreate')
tree = rt.TTree("BinInfo", "BinInfo")
name = bytearray(1)
duration = np.array([0], dtype=np.float64)
totalDuration = np.array([0], dtype=np.float64)
startMJD = np.array([0], dtype=np.float64)
stopMJD = np.array([0], dtype=np.float64)

# 将变量与branch关联
tree.Branch("name", name, "name/s")
tree.Branch("duration", duration, "duration/D")
tree.Branch("totalDuration", totalDuration, "totalDuration/D")
tree.Branch("startMJD", startMJD, "startMJD/D")
tree.Branch("stopMJD", stopMJD, "stopMJD/D")

# 填充数据
nside=1024
npix=hp.nside2npix(nside)
dtype1 = [('count', float)]
dtype2 = [('count', float)]
skymaponout=np.zeros(npix, dtype = dtype1)
skymapoffout=np.zeros(npix, dtype = dtype2)
pixid = np.arange(npix)
pixarea= 4*np.pi/npix


In [ ]:
for i in range(14):
    name[0]=i
    duration[0] = 10
    totalDuration[0] = header_value[1]*24 #*0.99726956
    startMJD[0]=58844.0
    stopMJD[0]=60156.0
    tree.Fill()
fout.Write()
fout.Close()

In [ ]:
for i in range(14):
    print('nHit%02d'%int(i))
    fout = rt.TFile.Open(f"~/Science/KM2A1234full_skymap.root", 'UPDATE')
    #skymap = file[f"hon_{i}"].values().T
    skymap = histogram_on.values()[:,:,i].T
    #plt.imshow(skymap, origin="lower", extent=[0,360,-20,80], aspect=True)
    skymapoff = histogram_off.values()[:,:,i].T
    ra = histogram_on.axis("x").centers()
    dec = histogram_on.axis("y").centers()
    RA, DEC=np.meshgrid(ra,dec)
    # plt.imshow(skymap, origin="lower", extent=[0,360,-90,90], aspect=True)
    fon = interp2d(ra,dec,skymap,kind='linear')
    fbkg = interp2d(ra,dec,skymapoff,kind='linear')
    
    ndir=fout.mkdir('nHit%02d'%int(i),'nHit%02d'%int(i))


    tree1=rt.TTree('data','data')
    tree2=rt.TTree('bkg','bkg')
    obj1 = rt.TParameter(int)("Nside",1024)
    obj2 = rt.TParameter(int)("Scheme",0)

    tree1.SetDirectory(ndir)
    tree1.SetEntries(npix)
    tree2.SetDirectory(ndir)
    tree2.SetEntries(npix)

    tree1.GetUserInfo().Add(obj1)
    tree1.GetUserInfo().Add(obj2)
    tree2.GetUserInfo().Add(obj1)
    tree2.GetUserInfo().Add(obj2)

    for ipix in tqdm(pixid):
        ra_pix , dec_pix = hp.pix2ang(1024,ipix,lonlat=True)
        if (dec_pix<-20.) | (dec_pix>80.):
            skymaponout[ipix]=hp.UNSEEN
            skymapoffout[ipix]=hp.UNSEEN
            continue
        skymaponout[ipix]=fon(ra_pix,dec_pix)/(np.radians(0.1)*np.radians(0.1)*np.cos(np.radians(dec_pix)))*pixarea
        skymapoffout[ipix]=fbkg(ra_pix,dec_pix)/(np.radians(0.1)*np.radians(0.1)*np.cos(np.radians(dec_pix)))*pixarea
    rn.array2tree(skymaponout,tree=tree1)
    rn.array2tree(skymapoffout,tree=tree2)
    fout.Write()
    fout.Close()

In [ ]:
import sys
sys.path.append("/data/home/rcy/data/km2a_data/3MLWCDA-master/Standard/src")
import mylib as my

In [ ]:
%matplotlib inline
map2, skymapHeader = hp.read_map("/data/home/rcy/data/km2a_data/mysig/KM2A_all_final_nHit000_0.69.fits.gz",h=True)
map2=hp.ma(map2)
my.hpDraw("region_name", "Modelname", map2,0,0,skyrange=(0,360,-20,80),
            colorlabel="Significance", contours=[10000000], save=False, cat={}, color="Milagro", xsize=2048)

In [ ]:
map2, skymapHeader = hp.read_map("/data/home/rcy/data/km2a_data/sg_data_nHit000_0.25.fits.gz",h=True, field=0)
map2=hp.ma(map2)
my.hpDraw("region_name", "Modelname", map2,0,0,skyrange=(0,360,-20,80),
            colorlabel="Significance", contours=[10000000], save=False, cat={}, color="Milagro", xsize=2048)

In [ ]:
29189.7/20479.6

In [ ]:
87.62*1.4253061583234048

In [ ]:
import array
import ctypes
file_chnage = rt.TFile.Open(f"/data/home/cwy/Science/3MLWCDA/Standard/src/test.root", 'UPDATE')
tree = file_chnage.Get("BinInfo")

newtree = tree.CloneTree(0)
# name = array.array('d', [0])
# name = rt.TString()
# name = rt.std.string()
duration = array.array('d', [0])
totalDuration = array.array('d', [0])
# name = array.array('b', [0])
name = bytearray(1)
# 19103.1
newtree.SetBranchAddress("duration", duration)
newtree.SetBranchAddress("name", name)
newtree.SetBranchAddress("totalDuration", totalDuration)
# b = newtree.GetBranch("name")
# newtree.GetListOfBranches().Remove(b)
# newtree.Branch("name", name, 'name/C')  #rt.AddressOf(

for i in tqdm(range(14)):
    tree.GetEntry(i)
    newtree.GetEntry(i)
    print(i, name[0])
    duration[0] = 10
    totalDuration[0]=710.746617
    # name.assign(str(i))
    name[0]=i
    # name.SetString(str(i))
    newtree.Fill()
print("!")
file_chnage.Write("", rt.TObject.kOverwrite)
file_chnage.Close()

In [ ]:
data_position="/data/home/rcy/data/km2a_data/data.root"
file = uproot.open(data_position)
file["header"].values()
# header_value_on[0]

In [ ]:
import array
import ctypes
file_chnage = rt.TFile.Open(f"/data/home/rcy/data/km2a_data/sg_data.root", 'UPDATE')
tree = file_chnage.Get("BinInfo")

newtree = tree.CloneTree(0)
# name = array.array('d', [0])
# name = rt.TString()
# name = rt.std.string()
duration = array.array('d', [0])
totalDuration = array.array('d', [0])
# name = array.array('b', [0])
name = bytearray(1)
newtree.SetBranchAddress("duration", duration)
newtree.SetBranchAddress("name", name)
newtree.SetBranchAddress("totalDuration", totalDuration)
# b = newtree.GetBranch("name")
# newtree.GetListOfBranches().Remove(b)
# newtree.Branch("name", name, 'name/C')  #rt.AddressOf(

for i in tqdm(range(14)):
    tree.GetEntry(i)
    newtree.GetEntry(i)
    print(i, name[0])
    duration[0] = 10
    totalDuration[0]=1216.239092*24
    # name.assign(str(i))
    name[0]=i
    # name.SetString(str(i))
    newtree.Fill()
print("!")
file_chnage.Write("", rt.TObject.kOverwrite)
file_chnage.Close()